In [1]:
import pandas as pd
import glob
import os

def process_file(file_path):
    try:
        # Membaca metadata
        head = pd.read_excel(file_path, usecols="C", nrows=5, header=None)
        head.iloc[:, 0] = head.iloc[:, 0].str.replace(": ", "", regex=False)

        station_id = head.iloc[0, 0]
        station_name = head.iloc[1, 0]
        latitude = float(head.iloc[2, 0])
        longitude = float(head.iloc[3, 0])
        elevation = float(head.iloc[4, 0])

        # Membaca data, termasuk kolom tanggal
        data = pd.read_excel(file_path, usecols="A:K", skiprows=8, nrows=365)

        # Pastikan data tidak kosong
        if data.empty:
            print(f"File {file_path} tidak memiliki data.")
            return None

        # Mengambil tanggal dari kolom A
        dates = data.iloc[:, 0]

        # Periksa apakah ada tanggal kosong atau duplikat
        if dates.isnull().any():
            print(f"File {file_path} memiliki tanggal kosong.")
            return None

        if dates.duplicated().any():
            print(f"File {file_path} memiliki tanggal duplikat.")
            return None

        # Mengambil data relevan setelah baris "Keterangan :"
        keterangan_index = data[data.iloc[:, 0].str.contains("Keterangan : ", na=False)].index
        if not keterangan_index.empty:
            last_row_index = keterangan_index[0] - 4
            relevant_data = data.loc[last_row_index:]
        else:
            relevant_data = data

        # Transpose data
        data_transposed = relevant_data.T

        # Mengambil header dan data Tavg
        headers = data_transposed.iloc[0].values
        tavg_data = data_transposed.iloc[3].values
        rr_data = data_transposed.iloc[5].values
        ff_x_data = data_transposed.iloc[7].values
        ddd_x_data = data_transposed.iloc[8].values

        # Membuat DataFrame untuk data Tavg
        tavg_df = pd.DataFrame([tavg_data])
        rr_df = pd.DataFrame([rr_data])
        ff_x_df = pd.DataFrame([ff_x_data])
        ddd_x_df = pd.DataFrame([ddd_x_data])

        # Membuat DataFrame metadata
        metadata = {
            'station_id': [station_id],
            'station_name': [station_name],
            'latitude': [latitude],
            'longitude': [longitude],
            'elevation': [elevation]
        }
        metadata_df = pd.DataFrame(metadata)

        # Menambahkan tanggal sebagai kolom
        tavg_df.columns = dates.values  # Menggunakan dates sebagai header kolom
        rr_df.columns = dates.values
        ff_x_df.columns = dates.values
        ddd_x_df.columns = dates.values

        # Menggabungkan metadata dan data Tavg
        combined_data_tavg = pd.concat([metadata_df, tavg_df], axis=1)
        combined_data_rr = pd.concat([metadata_df, rr_df], axis=1)
        combined_data_ff_x = pd.concat([metadata_df, ff_x_df], axis=1)
        combined_data_ddd_x = pd.concat([metadata_df, ddd_x_df], axis =1)

        return combined_data_tavg, combined_data_rr, combined_data_ff_x, combined_data_ddd_x

    except Exception as e:
        print(f"Error saat memproses file {file_path}: {e}")
        return None, None, None

# Menentukan pola file dengan wildcard
file_pattern = r'data\*.xlsx'
files = glob.glob(file_pattern)

# Filter file sementara Excel yang diawali '~$'
files = [file for file in files if not os.path.basename(file).startswith('~$')]

all_tavg_data = []
all_rr_data = []
all_ff_x_data = []
all_ddd_x_data = []


# Iterasi melalui daftar file
for file in files:
    tavg_df, rr_df, ff_x_df, ddd_x_df = process_file(file)
    if tavg_df is not None:
        all_tavg_data.append(tavg_df)
        all_rr_data.append(rr_df)
        all_ff_x_data.append(ff_x_df)
        all_ddd_x_data.append(ddd_x_df)

# Menggabungkan semua DataFrame jika ada data valid
if all_tavg_data:
    final_combined_tavg = pd.concat(all_tavg_data, ignore_index=True)
    final_combined_rr = pd.concat(all_rr_data, ignore_index=True)
    final_combined_ff_x = pd.concat(all_ff_x_data, ignore_index=True)
    final_combined_ddd_x = pd.concat(all_ddd_x_data, ignore_index=True)
    output_file = r'data_lengkap.xlsx'

    with pd.ExcelWriter(output_file) as writer:
        final_combined_tavg.to_excel(writer, sheet_name='Tavg', index=False) 
        final_combined_rr.to_excel(writer, sheet_name='RR', index =False)
        final_combined_ff_x.to_excel(writer, sheet_name='ff_x', index=False)
        final_combined_ddd_x.to_excel(writer, sheet_name='ddd_x', index=False)

# Menampilkan hasil
    print(f"Data Berhasil disimpan ke {output_file}")
else:
    print("Tidak ada data yang valid untuk digabungkan.")


Data Berhasil disimpan ke data_lengkap.xlsx
